# <font color='purple'>CoyneLab_iPSC_patients_TDP43_PB Preprocessing QC statistics </font> 
### October 2024

In [ ]:
import os
NOVA_HOME = '/home/labs/hornsteinlab/Collaboration/MOmaps_Noam/MOmaps'
NOVA_DATA_HOME = '/home/labs/hornsteinlab/Collaboration/MOmaps'

LOGS_PATH = os.path.join(NOVA_DATA_HOME, "outputs/preprocessing/AlyssaCoyne/logs/preprocessing_CoyneLab_iPSC_patients_TDP43_PB_7tiles/")
PLOT_PATH = os.path.join(NOVA_DATA_HOME, "outputs/preprocessing/AlyssaCoyne/logs/preprocessing_CoyneLab_iPSC_patients_TDP43_PB_7tiles/plots")

NOVA_HOME = '/home/labs/hornsteinlab/Collaboration/NOVA_Oz/NOVA'
NOVA_DATA_HOME = '/home/labs/hornsteinlab/Collaboration/MOmaps'
LOGS_PATH = os.path.join(NOVA_HOME, 'logs', 'Alyscoyn')
PLOT_PATH = os.path.join(NOVA_HOME, 'src', 'preprocessing', 'notebooks','figures','Alyscoyn_80pct')


os.chdir(NOVA_HOME)

import pandas as pd
import contextlib
import io
from IPython.display import display, Javascript
import seaborn as sns
from tools.preprocessing_tools.qc_reports.qc_utils import log_files_qc, run_validate_folder_structure, display_diff, sample_and_calc_variance, \
                                                show_site_survival_dapi_brenner, show_site_survival_dapi_cellpose, \
                                                show_site_survival_dapi_tiling, show_site_survival_target_brenner, \
                                                calc_total_sums, plot_filtering_heatmap, show_total_sum_tables, \
                                                plot_cell_count, plot_catplot, plot_hm_combine_batches, plot_hm, \
                                                run_calc_hist_new
                                                
from tools.preprocessing_tools.qc_reports.qc_config import AC_panels, AC_markers, AC_marker_info, AC_cell_lines, AC_cell_lines_to_cond,\
                                    AC_cell_lines_for_disp, AC_reps, AC_line_colors, AC_lines_order, AC_custom_palette,\
                                    AC_expected_dapi_raw, AC_cell_lines_to_reps
%load_ext autoreload
%autoreload 2

In [ ]:
df = log_files_qc(LOGS_PATH,only_wt_cond=True)
df_dapi = df[df.marker=='DAPI']
df_target = df[df.marker!='DAPI']

In [ ]:
# choose batches
batches = ['batch1']
batches

## Actual Files Validation

### Raw Files Validation

1. How many site **tiff** files do we have in each folder?
2. Are all existing files valid? (tif, at least 2049kB, not corrupetd)

In [ ]:

root_directory_raw = os.path.join(NOVA_DATA_HOME, 'input', 'images', 'raw', 'AlyssaCoyne', 'MOmaps_iPSC_patients_TDP43_PB_CoyneLab')


batches_raw = [batch.replace("_16bit_no_downsample","") for batch in batches]
raws = run_validate_folder_structure(root_directory_raw, False, AC_panels, AC_markers.copy(),PLOT_PATH, AC_marker_info,
                                    AC_cell_lines_to_cond, AC_reps, AC_cell_lines_for_disp, AC_expected_dapi_raw,
                                     batches=batches_raw, fig_width=5, cell_lines_to_reps=AC_cell_lines_to_reps,
                                    expected_count=10)

### Processed Files Validation

1. How many site **npy** files do we have in each folder? -> How many sites survived the pre-processing?
2. Are all existing files valid? (at least 100kB, npy not corrupted)

In [ ]:
root_directory_proc = os.path.join(NOVA_DATA_HOME, 'input', 'images', 'processed', 'AlyssaCoyne80pct', 'MOmaps_iPSC_patients_TDP43_PB_CoyneLab_7tiles')
procs = run_validate_folder_structure(root_directory_proc, True, AC_panels, AC_markers,PLOT_PATH, AC_marker_info,
                                    AC_cell_lines_to_cond, AC_reps, AC_cell_lines_for_disp, AC_expected_dapi_raw,
                                     batches=batches, fig_width=5, cell_lines_to_reps=AC_cell_lines_to_reps,
                                    expected_count=10)

### Difference between Raw and Processed

In [ ]:
display_diff(batches, raws, procs, PLOT_PATH, fig_width=5)

### Variance in each batch (of processed files)

In [ ]:

for batch in batches:
    with contextlib.redirect_stdout(io.StringIO()):
        var = sample_and_calc_variance(root_directory_proc, batch, 
                                       sample_size_per_markers=200, cond_count=1, rep_count=len(AC_reps), 
                                       num_markers=len(AC_markers))
    print(f'{batch} var: ',var)

## Preprocessing Filtering qc
By order of filtering

### 1. % site survival after Brenner on DAPI channel
Percentage out of the total sites

In [ ]:
dapi_filter_by_brenner = show_site_survival_dapi_brenner(df_dapi,batches, AC_line_colors, AC_panels,
                                                        figsize=(6,3), reps=AC_reps, vmax=10)

### 2. % Site survival after Cellpose
Percentage out of the sites that passed the previous filter. In parenthesis are absolute values.

**A site will be filtered out if Cellpose found 0 cells in it.**

In [ ]:
dapi_filter_by_cellpose = show_site_survival_dapi_cellpose(df_dapi, batches, dapi_filter_by_brenner, 
                                                           AC_line_colors, AC_panels, figsize=(6,3), reps=AC_reps)

### 3. % Site survival by tiling
Percentage out of the sites that passed the previous filter. In parenthesis are absolute values.

**A site will be filtered out if after tiling, no tile is containing at least 85% of a cell that Cellpose detected.**

In [ ]:
dapi_filter_by_tiling=show_site_survival_dapi_tiling(df_dapi, batches, dapi_filter_by_cellpose, 
                                                     AC_line_colors, AC_panels,figsize=(6,3), reps=AC_reps)

### 4. % Site survival after Brenner on target channel
Percentage out of the sites that passed the previous filter. In parenthesis are absolute values (if different than the percentages).

In [ ]:
show_site_survival_target_brenner(df_dapi, df_target, dapi_filter_by_tiling,
                                 figsize=(6,4), markers=AC_markers)

## Statistics About the Processed Files

In [ ]:
names = ['Total number of tiles', 'Total number of whole cells']
stats = ['n_valid_tiles','site_whole_cells_counts_sum','site_cell_count','site_cell_count_sum']
total_sum = calc_total_sums(df_target, df_dapi, stats, AC_markers)

### Total tiles

In [ ]:
total_sum.n_valid_tiles.sum()

### Total whole nuclei in tiles

In [ ]:
total_sum[total_sum.marker =='DAPI'].site_whole_cells_counts_sum.sum()

### Total nuclei in sites

In [ ]:
total_sum[total_sum.marker =='DAPI'].site_cell_count.sum()

In [ ]:
show_total_sum_tables(total_sum)

### Show Total Tile Counts
For each batch, cell line, replicate and markerTotal number of tiles

In [ ]:
to_heatmap = total_sum.rename(columns={'n_valid_tiles':'index'})
plot_filtering_heatmap(to_heatmap, extra_index='marker', vmin=None, vmax=None,
                      xlabel = 'Total number of tiles', show_sum=True, figsize=(6,5))

### Show Total Whole Cell Counts
For each batch, cell line, replicate and markerTotal number of tiles

In [ ]:
to_heatmap = total_sum.rename(columns={'site_whole_cells_counts_sum':'index'})
plot_filtering_heatmap(to_heatmap, extra_index='marker', vmin=None, vmax=None,
                      xlabel = 'Total number of whole cells', show_sum=True, figsize=(6,5))

### Show **Cell Count** Statistics per Batch

In [ ]:
df_no_empty_sites = df_dapi[df_dapi.n_valid_tiles !=0]
plot_cell_count(df_no_empty_sites, AC_lines_order, AC_custom_palette, y='site_cell_count_sum', 
                title='Cell Count Average per Site (from tiles)', figsize=(8,3))

plot_cell_count(df_no_empty_sites, AC_lines_order, AC_custom_palette, y='site_whole_cells_counts_sum',
                title='Whole Cell Count Average per Site',figsize=(8,3))

plot_cell_count(df_no_empty_sites, AC_lines_order, AC_custom_palette, y='site_cell_count',
               title='Cellpose Cell Count Average per Site',figsize=(8,3))


### Show **Tiles** per Site Statistics


In [ ]:
df_dapi.groupby(['cell_line_cond']).n_valid_tiles.mean()

In [ ]:
plot_catplot(df_dapi, sns.color_palette('colorblind'), reps=AC_reps,x='cell_line', y_title='Valid Tiles Count', x_title='Cell Line', y='n_valid_tiles', hue='rep',
             height=4, aspect=2)


### Show Mean of cell count in valid tiles

In [ ]:
plot_hm(df_dapi, split_by=None,rows='cell_line', columns='rep')

# Assessing Staining Reproducibility and Outliers

In [ ]:
for batch in batches:
    print(batch)
    run_calc_hist_new(batch,AC_cell_lines_for_disp, AC_markers, root_directory_raw, root_directory_proc,
                           hist_sample=10,sample_size_per_markers=10, ncols=4, nrows=1, figsize=(6,2))
    print("="*30)
    

In [ ]:
# save notebook as HTML ( the HTML will be saved in the same folder the original script is)
display(Javascript('IPython.notebook.save_checkpoint();'))
os.system(f'jupyter nbconvert --to html tools/preprocessing_tools/qc_reports/qc_report_CoyneLab_iPSC_patients_80pct.ipynb --output {NOVA_HOME}/manuscript/preprocessing_qc_reports/qc_report_CoyneLab_iPSC_patients_80pct.html')